# Embedding's for MovieLens dataset

## Описание задачи

Необходимо построить векторное представление пользователей и фильмов используя нейросетевые подходы, чтобы можно было по эмбендингу пользователя искать похожие эмбендинги фильмов и рекомендовать ему их.

Обратить внимание на:

1. Какие данных на обучение и валидацию, обоснование.
2. Выбор и обоснование метрики
3. Разработка архитектуры нейронной сети с пояснением
4. Обучение и валидация
5. Решение должно быть воспроизводимым с подробными комментариями на каждом шаге

## Описание данных

*Полный* датасет MovieLens с сайта grouplens.org: https://files.grouplens.org/datasets/movielens/ml-latest.zip

В датасете содержится информация о фильмах, оценках пользователей и тегах фильмов.


Структура файла данных рейтингов (ratings.csv)
-----------------------------------------

Все рейтинги содержатся в файле `ratings.csv`. Каждая строка этого файла после строки заголовка представляет собой одну оценку одного фильма одним пользователем и имеет следующий формат:

    userId,movieId,rating,timestamp

Строки в этом файле упорядочены сначала по идентификатору пользователя, затем, внутри пользователя, по идентификатору фильма.

Оценки выставляются по 5-звездочной шкале, с шагом в ползвезды (0,5 звезды - 5,0 звезды).

Временные метки представляют собой секунды с полуночи по всемирному координированному времени (UTC) от 1 января 1970 года.


Структура файла данных тегов (tags.csv)
-----------------------------------

Все теги содержатся в файле `tags.csv`. Каждая строка этого файла после строки заголовка представляет собой один тег, примененный к одному фильму одним пользователем, и имеет следующий формат:

    userId,movieId,tag,timestamp

Строки в этом файле упорядочены сначала по userId, затем, внутри пользователя, по movieId.

Теги - это созданные пользователем метаданные о фильмах. Каждый тег обычно представляет собой одно слово или короткую фразу. Смысл, значение и цель конкретного тега определяется каждым пользователем.

Временные метки представляют собой секунды с полуночи по всемирному координированному времени (UTC) от 1 января 1970 года.


Структура файла данных фильмов (movies.csv)
---------------------------------------

Информация о фильмах содержится в файле `movies.csv`. Каждая строка этого файла после строки заголовка представляет один фильм и имеет следующий формат:

    movieId,title,genres

Названия фильмов вводятся вручную или импортируются из <https://www.themoviedb.org/> и включают год выпуска в круглых скобках. В этих названиях могут быть ошибки и несоответствия.

Жанры представляют собой список, разделенный трубкой, и выбираются из следующих:

* боевик
* Приключения
* Анимация
* Детские
* Комедия
* Криминал
* Документальный
* Драма
* Фэнтези
* Фильм-нуар
* Ужасы
* Мюзикл
* Мистерия
* Романтика
* Научная фантастика
* Триллер
* Война
* Вестерн
* (жанры не указаны)


Структура файла данных ссылок (links.csv)
---------------------------------------

Идентификаторы, которые могут быть использованы для ссылок на другие источники данных о фильмах, содержатся в файле `links.csv`. Каждая строка этого файла после строки заголовка представляет один фильм и имеет следующий формат:

    movieId,imdbId,tmdbId

movieId - это идентификатор для фильмов, используемых <https://movielens.org>. Например, фильм "История игрушек" имеет ссылку <https://movielens.org/movies/1>.

imdbId - это идентификатор для фильмов, используемых <http://www.imdb.com>. Например, фильм "История игрушек" имеет ссылку <http://www.imdb.com/title/tt0114709/>.

tmdbId - идентификатор для фильмов, используемых <https://www.themoviedb.org>. Например, фильм "История игрушек" имеет ссылку <https://www.themoviedb.org/movie/862>.

Использование перечисленных выше ресурсов регулируется условиями каждого поставщика.


Геном тегов (genome-scores.csv и genome-tags.csv)
-------------------------------------------------

Этот набор данных включает текущую копию генома тегов.

[genome-paper]: http://files.grouplens.org/papers/tag_genome.pdf

Геном тегов - это структура данных, которая содержит оценки релевантности тегов для фильмов.  Структура представляет собой плотную матрицу: каждый фильм в геноме имеет значение для *каждого* тега в геноме.

Как описано в [этой статье][genome-paper], геном тегов кодирует, насколько сильно фильмы проявляют определенные свойства, представленные тегами (атмосферность, заставляющие задуматься, реалистичность и т.д.). Геном тегов был вычислен с помощью алгоритма машинного обучения на пользовательском контенте, включая теги, рейтинги и текстовые рецензии.

Геном разделен на два файла.  Файл `genome-scores.csv` содержит данные о релевантности тегов фильма в следующем формате:

    movieId,tagId,relevance

Второй файл, `genome-tags.csv`, содержит описания тегов для идентификаторов тегов в файле генома в следующем формате:

    tagId,tag

Значения `tagId` генерируются при экспорте набора данных, поэтому они могут отличаться в разных версиях наборов данных MovieLens.

При ссылке на геномные данные тегов, пожалуйста, указывайте следующую цитату:

> Jesse Vig, Shilad Sen, and John Riedl. 2012. The Tag Genome: Кодирование знаний сообщества для поддержки новых взаимодействий. ACM Trans. Interact. Intell. Syst. 2, 3: 13:1-13:44. <https://doi.org/10.1145/2362394.2362395>

## Анализ и подготовка данных

In [46]:
import warnings
warnings.filterwarnings('ignore')

# data visualisation and manipulation
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import style
import seaborn as sns

#configure
# sets matplotlib to inline and displays graphs below the corressponding cell.
%matplotlib inline
style.use('fivethirtyeight')
sns.set(style='whitegrid',color_codes=True)

#model selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.metrics import accuracy_score,precision_score,recall_score,confusion_matrix,roc_curve,roc_auc_score
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder

#preprocess.
from keras.preprocessing.image import ImageDataGenerator

#dl libraraies
import keras
from keras import backend as K
from keras.models import Sequential
from keras.layers import Dense , Concatenate, Dot
from keras.optimizers import Adam,SGD,Adagrad,Adadelta,RMSprop
from keras.utils import to_categorical
from keras.utils.vis_utils import model_to_dot
from keras.callbacks import ReduceLROnPlateau, EarlyStopping
from keras.models import Model


# specifically for deeplearning.
from keras.layers import Dropout, Flatten,Activation,Input,Embedding
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
import tensorflow as tf
import random as rn
from IPython.display import SVG

In [127]:
movies = pd.read_csv('ml-latest/movies.csv', dtype={'movieId': 'int32', 'title': 'str', 'genres': 'str'})
ratings = pd.read_csv('ml-latest/ratings.csv', dtype={'userId': 'int32', 'movieId': 'int32', 'rating': 'float16', 'timestamp': 'int32'})

In [128]:
movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [129]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264


In [130]:
# merge ratings and movies
df = pd.merge(ratings, movies, on='movieId')
df.head()

,userId,movieId,rating,timestamp,title,genres
0,1,307,3.5,1256677221,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
1,6,307,4.0,832059248,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
2,56,307,4.0,1383625728,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
3,71,307,5.0,1257795414,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama
4,84,307,3.0,999055519,Three Colors: Blue (Trois couleurs: Bleu) (1993),Drama


In [131]:
df = df.sample(n=500000, random_state=42)

In [132]:
del movies, ratings

In [133]:
# rename columns userId -> user_id, movieId -> movie_id
df.rename(columns={'userId':'user_id', 'movieId':'movie_id'}, inplace=True)

In [134]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500000 entries, 26261418 to 2367070
Data columns (total 6 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   user_id    500000 non-null  int32  
 1   movie_id   500000 non-null  int32  
 2   rating     500000 non-null  float16
 3   timestamp  500000 non-null  int32  
 4   title      500000 non-null  object 
 5   genres     500000 non-null  object 
dtypes: float16(1), int32(3), object(2)
memory usage: 18.1+ MB


In [135]:
df.isna().sum()

user_id      0
movie_id     0
rating       0
timestamp    0
title        0
genres       0
dtype: int64

In [136]:
# df.duplicated().sum()

0

In [137]:
df = df.drop(['title', 'genres'], axis=1)

In [138]:
rating_pre_user = df.groupby('user_id')['rating'].count()
rating_pre_movie = df.groupby('movie_id')['rating'].count()

print(f'Количество пользователей: {len(rating_pre_user)}')
print(f'Количество фильмов: {len(rating_pre_movie)}')
print()

print(f'Максимальная оценка: {df["rating"].max()}')
print(f'Минимальная оценка: {df["rating"].min()}')
print(f'Медианная оценка: {df["rating"].median()}')
print(f'Количество оценок: {df["rating"].count()}')
print()

print(f'Максимальное количество оценок на пользователя: {rating_pre_user.max()}')
print(f'Минимальное количество оценок на пользователя: {rating_pre_user.min()}')
print(f'Медианное количество оценок на пользователя: {rating_pre_user.median()}')
print()

print(f'Максимальное количество оценок на фильм: {rating_pre_movie.max()}')
print(f'Минимальное количество оценок на фильм: {rating_pre_movie.min()}')
print(f'Медианное количество оценок на фильм: {rating_pre_movie.median()}')

del rating_pre_user, rating_pre_movie

Количество пользователей: 142055
Количество фильмов: 17256

Максимальная оценка: 5.0
Минимальная оценка: 0.5
Медианная оценка: 3.5
Количество оценок: 500000

Максимальное количество оценок на пользователя: 406
Минимальное количество оценок на пользователя: 1
Медианное количество оценок на пользователя: 2.0

Максимальное количество оценок на фильм: 1741
Минимальное количество оценок на фильм: 1
Медианное количество оценок на фильм: 3.0


In [139]:
df[['user_id', 'movie_id', 'rating']].max()

user_id     283228.0
movie_id    193872.0
rating           5.0
dtype: float64

In [140]:
df.min()

user_id              2.0
movie_id             1.0
rating               0.5
timestamp    822873600.0
dtype: float64

In [141]:
df['movie_id'].nunique()

17256

In [142]:
df['user_id'].nunique()

142055

In [143]:
# show count of user with rating more than 20
print(df['user_id'].value_counts()[df['user_id'].value_counts() > 20].sum())
print(len(df))

68431
500000


In [119]:
# get only users with more than 15 ratings
# df = df[df['user_id'].isin(df['user_id'].value_counts()[df['user_id'].value_counts() > 15].index)]

In [144]:
df['user_id'] = df['user_id'].astype('category').cat.codes.values
df['movie_id'] = df['movie_id'].astype('category').cat.codes.values

In [145]:
df['user_id'].value_counts(ascending=True)

user_id
84859       1
137701      1
36995       1
78074       1
133186      1
         ... 
106574    136
121885    141
67322     152
58811     162
61575     406
Name: count, Length: 142055, dtype: int64

In [146]:
df.nunique()

user_id      142055
movie_id      17256
rating           10
timestamp    493651
dtype: int64

In [147]:
# create utility matrix for users and movies
utility_matrix = df.pivot_table(values='rating', index='user_id', columns='movie_id')
utility_matrix.head()

ValueError: negative dimensions are not allowed

In [ ]:
# define function to make encoding of genres
def ohe_genres(data, column_name='genres', split_symbol='|'):
    genres = data[column_name].str.split(split_symbol)
    genres = genres.explode().unique()
    genres = np.sort(genres)
    genres = genres[genres != '(no genres listed)']

    for genre in genres:
        data[genre] = data[column_name].str.contains(genre).astype(int)

    data.drop(column_name, axis=1, inplace=True)
    return data


In [ ]:
# df = ohe_genres(df)

In [ ]:
def get_last_n_ratings_by_user(
        df, n, min_ratings_per_user=1, user_colname="user_id", timestamp_colname="timestamp"
):
    return (
        df.groupby(user_colname)
        .filter(lambda x: len(x) >= min_ratings_per_user)
        .sort_values(timestamp_colname)
        .groupby(user_colname)
        .tail(n)
        .sort_values(user_colname)
    )

In [ ]:
def mark_last_n_ratings_as_validation_set(
        df, n, min_ratings=1, user_colname="user_id", timestamp_colname="timestamp"
):
    """
    Отмечает n последних по времени оценок, что включает их в проверочную выборку.
    Делается это путём добавления дополнительного столбца 'is_valid' в df.
    :param df: объект DataFrame, содержащий оценки, данные пользователем
    :param n: количество оценок, которые надо включить в проверочную выборку
    :param min_ratings: включать лишь пользователей, имеющих более этого количества оценок
    :param user_id_colname: имя столбца, содержащего идентификатор пользователя
    :param timestamp_colname: имя столбца, содержащего отметку времени
    :return: тот же df, в который добавлен дополнительный столбец 'is_valid'
    """
    df["is_valid"] = False
    df.loc[
        get_last_n_ratings_by_user(
            df,
            n,
            min_ratings,
            user_colname=user_colname,
            timestamp_colname=timestamp_colname,
        ).index,
        "is_valid",
    ] = True
    return df

In [ ]:
mark_last_n_ratings_as_validation_set(df, n=8, min_ratings=20);

In [ ]:
df = df.drop('timestamp', axis=1)

In [ ]:
train_df = df[df['is_valid'] == False].drop(columns=['is_valid'])
test_df = df[df['is_valid'] == True].drop(columns=['is_valid'])

In [ ]:
# show ratio of train and test data
print(f'Количество строк в тренировочном датасете: {len(train_df)}')
print(f'Количество строк в тестовом датасете: {len(test_df)}')
print(f'Отношение тренировочного датасета к тестовому: {len(train_df) / len(test_df)}')

## Построение модели